# Purpose

The purpose of this notebook is to run an example experiment using the functions from `fsnet-tools.py` to illustrate it's intended use.

# Data

A wide toy dataset will be generated using `make_regression` from `sklearn`.  I'll then split off a test dataset, standardize, and split off a validation set.

In [1]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# generate example regression dataset
X, y = make_regression(n_samples=500, n_features=4000, n_informative=25)

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2)


In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_full_scaled = scaler.fit_transform(X_train_full)
X_test = scaler.transform(X_test)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full_scaled, y_train_full)

print("train shape - X:", X_train.shape, ", y:", y_train.shape)
print("valid shape - X:", X_valid.shape, ", y:", y_valid.shape)
print("test shape - X:", X_test.shape, ", y:", y_test.shape)


train shape - X: (300, 4000) , y: (300,)
valid shape - X: (100, 4000) , y: (100,)
test shape - X: (100, 4000) , y: (100,)


# Build Model

Now I'll build the model using the newly created `fsnet-tools`.  `u_train` and `alpha` should be generated first, using the respective functions.  They will only need to be regenerated if `X_train` changes.  This is important to note, say, inside a cross-fold validation experiment.  

It's also important that hyperparameters match between the various function calls.  Here, I'm defining a hyperparameter dictionary, as this is consistent with my typical model experiment workflow.  This method also lends itself well to hyperparameter tuning via optimization algorithms.

In [3]:
from fsnet_tools import get_utrain
from fsnet_tools import get_alpha
from fsnet_tools import build_model

hps = {
    "bins": 10,
    "min_temp": 0.01,
    "start_temp": 10.0,
    "num_epochs": 500,
    "nfeat": 50,
    "h_size": 16,
}

u_train = get_utrain(X_train, bins=hps["bins"])
alpha = get_alpha(
    X_train,
    min_temp=hps["min_temp"],
    start_temp=hps["start_temp"],
    num_epochs=hps["num_epochs"],
)
model = build_model(
    num_inputs=X_train.shape[1],
    nfeat=hps["nfeat"],
    u_train=u_train,
    alpha=alpha,
    h_size=hps["h_size"],
    bins=hps["bins"],
    start_temp=hps["start_temp"],
    min_temp=hps["min_temp"],
    num_epochs=hps["num_epochs"],
)


Using TensorFlow backend.














Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



# Fit

Now I'll fit the data with the model defined above.

In [4]:
history = model.fit(
    X_train,
    {"recon": X_train, "reg_output": y_train},
    validation_data=(X_valid, {"recon": X_valid, "reg_output": y_valid}),
    epochs=hps["num_epochs"],
    verbose=1,
)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 300 samples, validate on 100 samples
Epoch 1/500
300/300 [==============================] - 1s 5ms/step - loss: 97668.2273 - recon_loss: 1.0018 - reg_output_loss: 97568.0448 - recon_mean_squared_error: 1.0018 - reg_output_mean_squared_error: 97568.0448 - val_loss: 86327.6764 - val_recon_loss: 1.2337 - val_reg_output_loss: 86204.3052 - val_recon_mean_squared_error: 1.2337 - val_reg_output_mean_squared_error: 86204.3052
Epoch 2/500
300/300 [==============================] - 0s 230us/step - loss: 97659.1739 - recon_loss: 1.0033 - reg_output_loss: 97558.8434 - recon_mean_squared_error: 1.0033 - reg_output_mean_squared_error: 97558.8434 - val_loss: 86331.0055 - val_recon_loss: 1.3519 - val_reg_output_loss: 86195.8194 - val_recon_mean_squared_error: 1.3519 - val_reg_output_mean_squared_error: 86195.8194
Epoch 3/500
300/300 [==============================] - 0s 223us/step - loss: 97650.9131

# Evaluate

Now I'll generate learning curves with `plotly`.  These are good for logging with `mlflow` or other similar experiment tracking libraries.

In [9]:
hist_df


,val_loss,val_recon_loss,val_reg_output_loss,val_recon_mean_squared_error,val_reg_output_mean_squared_error,loss,recon_loss,reg_output_loss,recon_mean_squared_error,reg_output_mean_squared_error,step
0,86327.676406,1.233706,86204.305156,1.233706,86204.305156,97668.227292,1.001812,97568.044792,1.001812,97568.044792,0
1,86331.005469,1.351861,86195.819375,1.351861,86195.819375,97659.173906,1.003305,97558.843385,1.003305,97558.843385,1
2,86392.361719,2.114701,86180.892344,2.114701,86180.892344,97650.913125,1.005332,97550.380885,1.005332,97550.380885,2
3,86299.638281,1.375389,86162.100156,1.375389,86162.100156,97638.557865,1.010385,97537.517812,1.010385,97537.517812,3
4,86263.851875,1.238314,86140.020937,1.238314,86140.020937,97622.903333,1.016711,97521.232917,1.016711,97521.232917,4
...,...,...,...,...,...,...,...,...,...,...,...
495,101707.158750,1.003143,101606.844375,1.003143,101606.844375,81911.124583,1.001488,81810.975625,1.001488,81810.975625,495
496,101595.818125,1.003940,101495.425312,1.003940,101495.425312,84210.004896,1.001169,84109.888333,1.001169,84109.888333,496
497,102940.080781,1.002808,102839.798594,1.002808,102839.798594,80349.549792,1.000879,80249.460521,1.000879,80249.460521,497
498,103772.033125,1.002582,103671.773438,1.002582,103671.773438,82465.125990,1.000567,82365.069115,1.000567,82365.069115,498


In [10]:
import pandas as pd

hist_df = pd.DataFrame(history.history)
hist_df["step"] = hist_df.index

plot_df = hist_df.loc[
    :,
    [
        "step",
        "loss",
        "val_loss",
        "reg_output_loss",
        "val_reg_output_loss",
        "recon_loss",
        "val_recon_loss",
    ],
]


In [12]:
plot_df_long = pd.melt(
    plot_df, id_vars="step", var_name="type", value_name="loss_value"
)


In [13]:
import plotly.express as px

fig = px.line(plot_df_long, x="step", y="loss_value", color="type", markers=True)
fig.show()


# Test

And now I'll evaluate the model on train, valid, and test sets.

In [8]:
print(
    "train MSE:",
    model.evaluate(X_train, {"recon": X_train, "reg_output": y_train}, verbose=0)[4],
)
print(
    "valid MSE:",
    model.evaluate(X_valid, {"recon": X_valid, "reg_output": y_valid}, verbose=0)[4],
)
print(
    "test MSE: ",
    model.evaluate(X_test, {"recon": X_test, "reg_output": y_test}, verbose=0)[4],
)


train MSE: 127814.26083333333
valid MSE: 102559.2309375
test MSE:  131554.17875
